In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict, namedtuple

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR, NuSVR
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from copy import copy
from math import sin, cos, tan, acos, atan
from sklearn.cluster import KMeans

from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

from scipy.optimize import minimize

In [2]:
PATH_TRAIN = "../data/train.csv"
PATH_TEST = "../data/test.csv"
PATH_SAVE = "sub_idao_example.csv"

In [3]:
sim_features = ['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']
features = ['x', 'y', 'z', 'Vx', 'Vy', 'Vz']

## Операции со времененем
sub_time - возвращает time1 - time2 в секундах

In [4]:
def time_to_int(times, start_time):
    return np.array([sub_time(time, start_time) for time in times])
        
def sub_time(time1, time2):
    time1 = datetime.strptime(time1, '%Y-%m-%dT%H:%M:%S.%f')
    time2 = datetime.strptime(time2, '%Y-%m-%dT%H:%M:%S.%f')
    
    month = time1.month - time2.month
    day = time1.day - time2.day
    hour = time1.hour - time2.hour
    minute = time1.minute - time2.minute
    second = time1.second - time2.second
    millisecond = (time1.microsecond - time2.microsecond) // 1000
    
    sec_mult = 1
    min_mult = 60  * sec_mult
    hour_mult = 60 * min_mult
    day_mult = 24 * hour_mult
    
    month_len = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    month_mult = sum(month_len[time2.month:time1.month]) * day_mult
    
    mults = np.array([sec_mult, min_mult, hour_mult, day_mult, month_mult])
    diffs = np.array([second, minute, hour, day, month])
    
    return sum(diffs * mults)

## Скейлит данные
Скейлит только время и реальные данные

In [5]:
TIME_COEF = 60 * 60 * 24 * 55


class Transformer():
    def __init__(self):
        self.scalers = [StandardScaler() for _ in range(6)]
        self.time_sc = StandardScaler()

    def fit_transform(self, data):
        X = data[sim_features + ['epoch']].to_numpy()
        y = data[features].to_numpy()
        
        self.start_time = X[0, 6]
        X[:, 6] = time_to_int(X[:, 6], self.start_time) / TIME_COEF
        
        for i in range(6):
            y[:, i] = self.scalers[i].fit_transform(y[:, i].reshape(-1, 1)).reshape(-1)
        return X, y
    
    def fit_transform_test(self, data_train, data_test):
        X_train = data_train[sim_features + ['epoch']].to_numpy()
        X_test = data_test[sim_features + ['epoch']].to_numpy()
        y = data_train[features].to_numpy()
        
        self.start_time = X_train[0, 6]
        self.time_sc.fit(np.hstack([time_to_int(X_train[:, 6], self.start_time), time_to_int(X_test[:, 6], self.start_time)]).reshape(-1, 1))
        
        X_train = self.transform(data_train)
        X_test  = self.transform(data_test)
        
        for i in range(6):
            self.scalers[i].fit(np.hstack([y[:, i], X_train[:, i], X_test[:, i]]).reshape(-1, 1))
            
            y[:, i] = self.scalers[i].transform(y[:, i].reshape(-1, 1)).reshape(-1)
        
        return X_train, X_test, y
        

    def transform(self, data):
        X = data[sim_features + ['epoch']].to_numpy()
        X[:, 6] = self.time_sc.transform(time_to_int(X[:, 6], self.start_time).reshape(-1, 1)).reshape(-1)
        return X
            

    def inverse_transform(self, y):
        for i in range(6):
            y[:, i] = self.scalers[i].inverse_transform(y[:, i].reshape(-1, 1)).reshape(-1)    
        return y

Делит numpy.array на две части

In [6]:
def split_data(X, y, train_size):
    index = int(len(X) * train_size)
    
    return X[:index], X[index:], y[:index], y[index:]

## Аппроксимация периодической функцией
Пока не используется

In [7]:
EPS = np.float64(1e-6)

class Xsin():
    def __init__(self, X_train, y_train, alpha=0.01):
        self.f = func(X_train.reshape(-1), y_train, alpha)
        self.g = grad(self.f)
        
        self.w = minimize(fun=self.f, x0=np.ones((8)), method='BFGS', jac=self.g, tol=TOL).x
        
    def predict(self, X_test):
        w = self.w
        return np.array([optimizing_f(self.w, x) for x in X_test.reshape(-1)])
       
        
def optimizing_f(w, x):
    return w[0] + w[1] * x + w[2] * sin(w[3] + w[4] * x) + w[5] * cos(w[6] + w[7] * x)

def func(X_train, y_train, alpha):
    def f(w):
        ans = []
        for x, y in zip(X_train, y_train):
            res = optimizing_f(w, x)
            #ans.append(abs(res - y) / (abs(res) + abs(y)) + alpha * w[1] ** 2)
            ans.append((res - y) ** 2 + alpha * np.linalg.norm(w) ** 2)
            
        return np.float64(np.mean(np.array(ans)))
    
    return f

def grad(f):
    def g(w):
        ans = []
        for i in range(8):
            d = np.float64(np.zeros((8)))
            d[i] = EPS
            
            ans.append((f(w + d) - f(w - d)) / (2 * EPS))

## Кластеризует траекторию одного спутника

In [8]:
def clusterize(data):
    times = list(data['epoch'])
    prev_time = times[0]
    cluster_index = 0
    clusters = [[] for _ in range(24)]
    clusters[0].append(0)
    
    for i in range(1, len(times)):
        if sub_time(times[i], prev_time) > 10:
            cluster_index = (cluster_index + 1) % 24
        
        clusters[cluster_index].append(i)
        prev_time = times[i]
            
    return [data.iloc[cluster] for cluster in clusters]

## Чтение и кластеризация трейновых данных

In [9]:
total_data = pd.read_csv(PATH_TRAIN, index_col=0)
test_data_full = pd.read_csv(PATH_TEST, index_col=0)
# total_data['id'] = total_data.index
# test_data_full['id'] = test_data_full.index

total_data['id'] = total_data.index
test_data_full['id'] = test_data_full.index

SAT_N = len(pd.unique(total_data["sat_id"]))

In [10]:
# df = pd.read_csv("../data_KSD_RP.csv", index_col=0)
# df['id'] = df.index
# # df.index.names = ['id']

# test_date = "2014-02-01T00:00:00.000"
# train_idxs = (df["epoch"] < test_date).values
# df_train = df[train_idxs]
# df_test = df[np.invert(train_idxs)]
# df_test_sgp4 = df_test[["id", "sat_id", "epoch", "x_sim", "y_sim", "z_sim", "Vx_sim", "Vy_sim", "Vz_sim"]]
# df_test_ans = df_test[["id", "sat_id", "x", "y", "z", "Vx", "Vy", "Vz"]]

# SAT_N = len(pd.unique(df_train["sat_id"]))

In [11]:
# total_data = df_train.drop(['RP', 'KSD'], axis=1)
# test_data_full = df_test_sgp4

In [12]:
data = total_data
clusters_list = []
for sat_id in range(SAT_N):
    clusters_list.append(clusterize(data[data['sat_id'] == sat_id]))

## Чтение и кластеризация тестовых данных (для трека А)

In [13]:
test_data_full.head()

,sat_id,epoch,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim,id
id,,,,,,,,,
215,0,2014-02-01T00:05:07.344,-28099.394464,-31393.873066,27879.580116,-2.048926,1.980606,-1.879962,215
216,0,2014-02-01T03:32:46.448,-42644.222149,-288.393217,-1087.275407,0.015348,2.814050,-2.577676,216
217,0,2014-02-01T07:00:25.552,-27979.382752,31096.467597,-29369.026222,2.033737,2.029262,-1.794169,217
218,0,2014-02-01T10:28:04.656,896.126549,49712.580025,-45492.481646,2.433120,1.022877,-0.859440,218
219,0,2014-02-01T13:55:43.760,30391.384369,58185.560870,-52301.500238,2.267209,0.390206,-0.285250,219


In [14]:
sat_ids = list(set(test_data_full['sat_id']))

In [15]:
test_clusters_list = []
for sat_id in range(SAT_N):
    if  sat_id not in sat_ids:
        test_clusters_list.append([])
    else:
        test_clusters_list.append(clusterize(test_data_full[test_data_full['sat_id'] == sat_id]))

## Перестановка тестовых кластеров в таком же порядке, как и трейновых

In [16]:
def reorder_clusters():
    for sat_id in sat_ids:
        reorder_clusters_id(sat_id)
        
def reorder_clusters_id(sat_id):
    train_clusters = clusters_list[sat_id]
    test_clusters = test_clusters_list[sat_id]
    test_cluster = test_clusters_list[sat_id][0]
        
    period = average_period(test_cluster)
    dists = [cluster_distance(train_cluster, test_cluster, period) for train_cluster in train_clusters]
        
    offset, dist = min(enumerate(dists), key=lambda x: x[1])
    
    assert dist < 2
        
    ordered_clusters = [test_clusters[(24 - offset + i) % 24] for i in range(24)]
    test_clusters_list[sat_id] = ordered_clusters
    
def cluster_distance(train_cluster, test_cluster, period):
    train_time = train_cluster['epoch'].to_numpy()[-1]
    test_time = test_cluster['epoch'].to_numpy()[0]
    
    return float_mod(train_time, test_time, period)

def float_mod(t1, t2, period):
    time_diff = sub_time(t2, t1)
    mod_value = int(time_diff / period)
    return min(time_diff - period * mod_value, period * (mod_value + 1) - time_diff)

def average_period(cluster):
    times = cluster['epoch'].to_numpy()
    periods = []
    for i in range(1, len(times)):
        time_dist = sub_time(times[i], times[i - 1])
        if time_dist > 1:
            periods.append(time_dist)
    
    return np.mean(np.array(periods))


def find_example(times):
    time1, time2 = times
    if sub_time(time2, time1) < 100:
        return 1
    else:
        return 0

reorder_clusters()

## Предсказание для пункта А

In [17]:
results = [[] for _ in range(7)]

def split(X, rate):
    index = int(len(X) * rate)
    return X[:index], X[index:]

for sat_id in tqdm(sat_ids):
    for cluster_index in range(24):
        test_data  = test_clusters_list[sat_id][cluster_index]
        train_data = clusters_list[sat_id][cluster_index]
        transformer = Transformer()
        X_train, X_test, y_train = transformer.fit_transform_test(train_data, test_data)
        
        rate = 0.5
        if (len(test_data) < 6):
            rate = 0.5
            
        X_test1, X_test2 = split(X_test, rate)
        
        _, X_train1 = split(X_train, 0.85)
        _, y_train1 = split(y_train, 0.85)
    
        model1 = make_pipeline(PolynomialFeatures(degree=3), Ridge(alpha=0.0002))
        model2 = Ridge(alpha=-0.001)
    
        for id_ in test_data['id']:
            results[0].append(id_)
        
        subm = np.empty((len(test_data), 6))
        X_train1_p = X_train1[:, [6]]
        X_test1_p = X_test1[:, [6]]
        
        X_train2_p = np.vstack([X_train1[:, [6]], X_test1_p])
        X_test2_p = X_test2[:, [6]]
        
        for feature_id in range(len(sim_features)):
            y = y_train1[:, feature_id]
            model1.fit(X_train1_p, y)
            
            y1 = model1.predict(X_test1_p)
            model2.fit(X_train2_p, np.hstack([y, y1]))
            
            y2 = model2.predict(X_test2_p)
        
            subm[:, feature_id] = np.hstack([y1, y2])
            
        subm = transformer.inverse_transform(subm)
        for feature_id in range(0, len(sim_features)):
            results[feature_id + 1] += list(subm[:, feature_id])
            

                
res_dict = {}
for i, feature in enumerate(features):
    res_dict[feature] = results[i + 1]
    
res_dict['id'] = results[0]
answer = pd.DataFrame(res_dict)

100%|██████████| 600/600 [03:32<00:00,  2.94it/s]


## Сохранение предсказаний в файл

In [18]:
tmp = answer.sort_values(by=['id']).set_index('id')
tmp.to_csv(PATH_SAVE, index=True)

## Сохранение последнего времени в кластере

In [19]:
first_times, last_times = [], []
sat_ids, cluster_ids = [], []

for i in range(SAT_N):
    for j in range(24):
        times = list(clusters_list[i][j]['epoch'])
        first_time = times[0]
        last_time = times[-1]
        
        first_times.append(first_time)
        last_times.append(last_time)
        sat_ids.append(i)
        cluster_ids.append(j)
        
result = {'sat_id' : sat_ids, 'cluster_id' : cluster_ids, 'last_time' : last_times}
result = pd.DataFrame.from_dict(result)

In [20]:
result.to_csv('cluster_last_time.csv')

## Тренировка модели для пункта B
Сохранение коэффициентов скейлинга для каждой feature в каждом кластере в формате строк 

(номер_спутника номер_кластера номер_фичи медиана отклонение)

scaled_x = (x - медиана) / отклонение


Сохранение массива коэффициентов для Ridge - свободный член, коэффициенты

In [21]:
def super_split(X):
    index = len(X) - 2
    
    while index >= 1 and ((X[index] - X[index - 1]) * (X[index + 1] - X[index]) > 0):
        index -= 1
        
    index = max(0.85 * len(X), index)
   # index = min(0.9 * len(X), index)
    index = int(index)
    
    return X[:index], X[index:]

In [22]:
model = Ridge(alpha=0.0003, tol=1e-8)

coefs = [[[0] * 6 for _ in range(24)] for _ in range(SAT_N)]
scalers = []

for sat_id in tqdm(range(SAT_N)):
    for cluster_id in range(24):
        train_data = clusters_list[sat_id][cluster_id]
        
        transformer = Transformer()
        X_train, y_train = transformer.fit_transform(train_data)
        
        sc = transformer.scalers
        
        means = [sc[i].mean_[0] for i in range(6)]
        scales = [sc[i].scale_[0] for i in range(6)]
        
        _, X_train_p = super_split(X_train[:, [6]])
        
        y_train_p = [0] * 6
        for feature_id in range(6):
            _, y_train_p[feature_id] = super_split(y_train[:, feature_id])
        
                                        
        for feature_id in range(6):
            y = y_train_p[feature_id]
            model.fit(X_train[-len(y):, [6]], y)
        
            coef = np.hstack([model.predict(np.array([[0]])), model.coef_])
            coefs[sat_id][cluster_id][feature_id] = coef
            scalers.append(' '.join(str(e) for e in [sat_id, cluster_id, feature_id, means[feature_id], scales[feature_id]]))

100%|██████████| 600/600 [01:33<00:00,  6.66it/s]


## Сохранение коэффициентов и скейлинга в файл

In [23]:
def save_to_file_scalers():
    with open('scalers.txt', 'w') as f:
        for line in scalers:
            f.write(line + '\n')

In [24]:
def save_to_file_coefs():
    with open('coefs.txt', 'w') as f:
        for sat_id in range(SAT_N):
            for cluster_id in range(24):
                for feature_id in range(6):
                    tmp_coefs = coefs[sat_id][cluster_id][feature_id]
                    #print(tmp_coefs)
                    #print(tmp_coefs[-1])
                    line = ' '.join([str(sat_id), str(cluster_id), str(feature_id)]) + ' ' + ' '.join(str(coef) for coef in tmp_coefs)
                    
                    f.write(line + '\n')

In [25]:
save_to_file_coefs()

In [26]:
save_to_file_scalers()